<a href="https://colab.research.google.com/github/willianszwy/Aprendizado-Supervisionado-no-Neuronio-Perceptron/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import numpy as np
import datetime as dt
import random

# Parte I - Resolvendo um problema linearmente separável

Clonando o repositório do github que contém o data

In [130]:
# Descomentar essa linha se der erro
#!git clone https://github.com/jesusdourado/rna-atividade2.1-data.git

In [131]:
exemplos = np.fromfile('rna-atividade2.1-data/data/dataAll.txt').reshape(-1,3)

In [132]:
random.seed(dt.datetime.now().time())

## Implementando o neurônio Perceptron de Rosenblatt

In [133]:
class Perceptron:

    def __init__(self, tamanho, bias = 0, taxa_aprendizado = 0.1, min_peso = -0.5, max_peso = 0.5) -> None:
        self.bias = bias
        self.taxa_aprendizado = taxa_aprendizado
        self.pesos = np.zeros(tamanho + 1)
        self.pesos[0] = random.uniform(min_peso, max_peso)
        self.pesos[1] = random.uniform(min_peso, max_peso)
        self.pesos[2] = random.uniform(min_peso, max_peso)
        self.entradas = np.empty(tamanho + 1)
        self.entradas[0] = self.bias
        self.erro = 0
        self.atualiza_pesos_contador = 0 


    def treinar(self, d):
        epoca = 0
        continua = True
        while continua:
            epoca += 1
            print("Época: ", epoca)
            contem_erro = False
            qtd_ajustes_pesos_epoca = 0

            for i in range(d.shape[0]):

                self.entradas[1] = d[i][0]
                self.entradas[2] = d[i][1]

                y = self.predicao()
                self.calcula_erro(y, d[i][2])

                if self.erro != 0:
                    self.atualiza_pesos()
                    qtd_ajustes_pesos_epoca += 1
                    contem_erro = True      

            print("Total ajustes de pesos da época: ", qtd_ajustes_pesos_epoca)

            if not contem_erro:
                continua = False
        
        print("Número total de ajustes dos pesos: ", self.atualiza_pesos_contador)
        print("Número de Épocas: ", epoca)

    def funcao_de_ativacao(self, x):
        return 1 if x >= 0 else 0

    def predicao(self):
        y = self.pesos.T.dot(self.entradas)
        return self.funcao_de_ativacao(y)

    def calcula_erro(self,y, yd):
        self.erro = yd - y

    def delta(self):
        return self.taxa_aprendizado * self.erro * self.entradas

    def atualiza_pesos(self):
        
            self.pesos =  self.pesos + self.delta()
            self.atualiza_pesos_contador += 1
            print("pesos atualizados: ",self.pesos)

In [134]:
neuronio = Perceptron(2)
neuronio.treinar(exemplos)

Época:  1
pesos atualizados:  [  0.20806421  36.17642203 -24.10646492]
pesos atualizados:  [   0.20806421  -17.48262797 -100.60221492]
pesos atualizados:  [  0.20806421  55.31339203 -64.77265492]
pesos atualizados:  [  0.20806421 137.91209203  15.08317508]
pesos atualizados:  [  0.20806421 143.75785203 -63.21141492]
pesos atualizados:  [   0.20806421  103.74894203 -141.10410492]
pesos atualizados:  [  0.20806421 157.41875203 -92.01183492]
pesos atualizados:  [   0.20806421  104.45869203 -148.88129492]
pesos atualizados:  [  0.20806421 164.01821203 -98.75896492]
pesos atualizados:  [   0.20806421  118.49358203 -164.39489492]
pesos atualizados:  [ 2.08064209e-01  2.09209442e+02 -8.96815749e+01]
pesos atualizados:  [   0.20806421  151.42845203 -149.98972492]
pesos atualizados:  [   0.20806421  103.16189203 -198.45944492]
pesos atualizados:  [   0.20806421  173.60924203 -156.34317492]
pesos atualizados:  [   0.20806421  126.76724203 -207.75304492]
pesos atualizados:  [   0.20806421  196.63

# Parte II - Experimentação

Calculando o identificador para ler o data**X**.txt

In [135]:
def caminhoDados ():
  matriculas = [1715080108, 1815310070, 1515080172, 1815310027]
  soma = 0

  for m in matriculas:
    soma += m % 10

  soma = soma % 4

  return 'rna-atividade2.1-data/data/data' + str(x) + '.txt'

print(caminhoDados())

rna-atividade2.1-data/data/data1.txt


In [136]:
exemplos = np.fromfile(caminhoDados()).reshape(-1, 3)
neuronio = Perceptron(2, -1, 0.1, -100, 100)
neuronio.treinar(exemplos)


Época:  1
pesos atualizados:  [ 30.13582833  24.84563034 -14.67315445]
pesos atualizados:  [ 30.23582833  24.73411675 -14.65324525]
pesos atualizados:  [ 30.33582833  24.62001215 -14.63567287]
pesos atualizados:  [ 30.23582833  24.71951664 -14.53677431]
pesos atualizados:  [ 30.13582833  24.81631941 -14.42447384]
pesos atualizados:  [ 30.03582833  24.93430105 -14.34381346]
pesos atualizados:  [ 29.93582833  25.0357061  -14.22513869]
pesos atualizados:  [ 29.83582833  25.15672993 -14.10909027]
pesos atualizados:  [ 29.73582833  25.24413    -13.9949335 ]
pesos atualizados:  [ 29.63582833  25.35685133 -13.89583051]
pesos atualizados:  [ 29.53582833  25.47492883 -13.79741796]
pesos atualizados:  [ 29.43582833  25.56680642 -13.71154024]
pesos atualizados:  [ 29.33582833  25.65701259 -13.6007575 ]
pesos atualizados:  [ 29.23582833  25.75538625 -13.50079489]
pesos atualizados:  [ 29.13582833  25.85585809 -13.41083308]
pesos atualizados:  [ 29.03582833  25.964913   -13.32097   ]
pesos atualiza

# Parte III - Validação *Holdout* em problema não linearmente separável